# Import Library

In [1]:
from utils import preprocessor
import pandas as pd
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
from utils import loss
from core import MusicRecognition
from utils import saver
from core import KNN

reloaded


# Load Dataset

In [ ]:
#load dataset
data = pd.read_csv('dataset/Emotion_features_Basshouse.csv')
data2 = pd.read_csv('dataset/Emotion_features_deephouse.csv')
data = data.append(data2, ignore_index=True)
data

# Feature Engineering to NP Array

In [2]:
data = pd.read_csv('dataset/Emotion_features.csv')
data = data.dropna()

In [3]:
#preprocess dataset
dataset_x, dataset_y = preprocessor.feature_engineering_knn_aidj_new(data)
dataset_x,dataset_y

(array([[ 4.42053126e+00,  2.60490923e+01],
        [ 6.77625058e+00,  3.30578880e+01],
        [ 2.23416521e-01,  3.18140298e+01],
        ...,
        [-2.92869632e+00,  3.54499028e+01],
        [-3.65868631e-02,  3.78521620e+01],
        [ 3.18483866e+00,  3.42546854e+01]]), array([[ 7., 14.,  4.],
        [ 6., 10.,  0.],
        [ 8., 19.,  0.],
        ...,
        [ 7.,  7.,  4.],
        [ 7., 18.,  4.],
        [ 8.,  2.,  4.]]))

# Split between train and test

In [4]:
#dataset splitting
x_train, x_test, y_train, y_test = train_test_split(dataset_x, dataset_y, test_size=0.1)
x_train

array([[ 3.25275377, 23.20198348],
       [ 4.36486959, 24.74449945],
       [ 5.99114456, 31.1609023 ],
       ...,
       [ 6.50768566, 23.17186985],
       [ 4.83779393, 21.32750801],
       [ 4.42812508, 26.88639441]])

# Reduce dimension using PCA
## Notes that the explained variance > 99% to make sure data persistence

In [6]:
#dimension reduction
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
import numpy as np
# pca = PCA(n_components=26)
scaler = StandardScaler()
scaler.fit(x_train)
# pca.fit(scaler.fit_transform(x_train)) #PCA based on dataset training only
# pca.explained_variance_ratio_.sum()

StandardScaler(copy=True, with_mean=True, with_std=True)

In [7]:
import matplotlib.pyplot as plt 
x_train_rd = (scaler.transform(x_train))
print(x_train_rd.shape)
datas = {'a': ((scaler.transform(dataset_x))).T[0],
        'b': ((scaler.transform(dataset_x))).T[1],
        'c': dataset_y.T[0]}
plt.scatter('a', 'b', c='c', data=datas)

(585, 2)


In [9]:
print(x_train)

[[1.35999178e+02 1.69507000e+05 1.28414394e+03 ... 2.67842177e+00
  1.09904917e+00 1.20790908e+00]
 [1.23046875e+02 1.40413000e+05 1.22098261e+03 ... 3.21596372e+00
  1.43576511e+00 2.06142145e+00]
 [1.29199219e+02 1.35430000e+05 1.16750000e+03 ... 2.37888435e+00
  1.23952970e+00 1.53643388e+00]
 ...
 [1.29199219e+02 1.59164000e+05 1.27331200e+03 ... 2.45202721e+00
  1.32962172e+00 1.76789393e+00]
 [1.29199219e+02 6.39080000e+04 8.08962025e+02 ... 3.23802268e+00
  1.81323130e+00 3.28780775e+00]
 [1.23046875e+02 1.57976000e+05 1.28435772e+03 ... 1.88778231e+00
  1.04438441e+00 1.09073880e+00]]


In [8]:
for neigh in range(100):
    knn = KNN.KNN(num_neighbor = neigh+1)
    # print(x_train)
    knn.train(x_train_rd, y_train)

    #yield acc
    y_pred = knn.infer((scaler.transform(x_test)))
    # print(y_pred.T[0])
    # print(loss.hamming_score(y_test, y_pred))
    print(accuracy_score(y_test.T[0], y_pred.T[0]))
    # print(accuracy_score(y_test.T[0], y_pred.T[0]) * accuracy_score(y_test.T[1], y_pred.T[1])) #ALL must TRUE metrics

0.4461538461538462
0.46153846153846156
0.5384615384615384
0.49230769230769234
0.5230769230769231
0.5076923076923077
0.49230769230769234
0.5384615384615384
0.5076923076923077
0.47692307692307695
0.49230769230769234
0.47692307692307695
0.49230769230769234
0.5076923076923077
0.5076923076923077
0.49230769230769234
0.5076923076923077
0.46153846153846156
0.46153846153846156
0.4461538461538462
0.47692307692307695
0.46153846153846156
0.47692307692307695
0.46153846153846156
0.4307692307692308
0.4307692307692308
0.4461538461538462
0.4461538461538462
0.4153846153846154
0.4153846153846154
0.36923076923076925
0.38461538461538464
0.38461538461538464
0.4307692307692308
0.4153846153846154
0.4153846153846154
0.4153846153846154
0.38461538461538464
0.4
0.38461538461538464
0.4
0.4
0.4
0.4
0.4
0.4
0.4153846153846154
0.4153846153846154
0.4153846153846154
0.4307692307692308
0.4307692307692308
0.4461538461538462
0.4307692307692308
0.4153846153846154
0.4153846153846154
0.4153846153846154
0.4307692307692308
0.4

In [9]:
#use best neighbor to train (only for class classifier)
knn = KNN.KNN(num_neighbor = 14)
    # print(x_train)
knn.train(x_train_rd, y_train.T[0])
y1_pred = knn.infer((scaler.transform(x_test)))
print(y1_pred)
    # print(loss.hamming_score(y_test, y_pred))
print(accuracy_score(y_test.T[0], y1_pred))

[6. 6. 6. 7. 7. 7. 7. 7. 6. 8. 6. 7. 7. 7. 7. 7. 8. 7. 8. 7. 7. 6. 7. 7.
 7. 7. 8. 7. 8. 8. 7. 6. 8. 7. 7. 7. 7. 7. 8. 8. 7. 6. 7. 8. 8. 7. 7. 7.
 7. 7. 7. 7. 8. 8. 7. 8. 8. 8. 7. 7. 7. 8. 7. 7. 8.]
0.5076923076923077


In [10]:
#linearRegression part (for energy regressor)
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import PolynomialFeatures
import numpy as np
lr = KNN.KNN(num_neighbor =13)
lr.train(x_train_rd, y_train.T[1])
y2_pred = lr.infer((scaler.transform(x_test)))
print(np.round(y2_pred))
print(accuracy_score(y_test.T[1], np.round(y2_pred)))
print(y_test.T[1])
print(np.square(y_test.T[1] - y2_pred).mean(axis=0))

[10.  2. 10. 10. 19. 10. 10. 19.  2. 10.  4. 10. 10.  2. 10. 10. 19. 10.
 19.  6. 10.  4. 10.  6.  0.  2. 10. 10. 10. 10.  2.  6. 10. 10.  2. 10.
 14. 10.  6. 19.  2.  0.  6. 19.  2. 10. 10. 10. 10. 10.  6. 10. 10. 10.
 10. 10.  2.  6.  6. 19. 18. 14. 14. 10.  2.]
0.16923076923076924
[14. 18. 18. 18. 18. 19. 19. 14. 16.  5. 12.  2.  5.  6. 14. 14. 14.  2.
 14.  1. 10.  6.  2. 10.  2. 10.  2. 10. 10.  7.  6. 12. 14. 10. 10. 10.
 10. 10. 10. 19. 18. 10.  2. 10.  7. 10. 14. 19. 18. 10. 19.  7.  6. 10.
 14.  9.  2.  2. 10. 14. 14. 16. 10. 14. 19.]
42.86153846153846


In [11]:
#linearRegression part (for energy regressor)
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import PolynomialFeatures
import numpy as np
lr = KNN.KNN(num_neighbor =6)
lr.train(x_train_rd, y_train.T[2])
y3_pred = lr.infer((scaler.transform(x_test)))
print(np.round(y3_pred))
print(accuracy_score(y_test.T[2], np.round(y3_pred)))

[2. 2. 1. 1. 0. 1. 4. 3. 4. 1. 4. 0. 1. 0. 3. 1. 1. 3. 1. 2. 1. 1. 3. 1.
 1. 1. 1. 0. 1. 1. 1. 4. 1. 0. 1. 1. 4. 0. 1. 0. 4. 2. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 0. 1. 1. 4. 1. 1. 4. 1. 1. 4. 1. 1. 0. 1.]
0.27692307692307694


In [ ]:
mr = MusicRecognition.MusicRecognition(knn, lr, pca)
saver.saveObj("cache/mr.bin",mr)

In [14]:
from sklearn.pipeline import make_pipeline
from sklearn.svm import SVC

svm = SVC(gamma='auto', kernel='rbf', degree=15)
    # print(x_train)
svm.fit(x_train_rd, y_train.T[1])
y1_pred = svm.predict((scaler.transform(x_test)))
print(y1_pred)
    # print(loss.hamming_score(y_test, y_pred))
print(accuracy_score(y_test.T[1], y1_pred))

[10. 10. 10. 10. 10. 10. 10. 10. 10. 10. 10. 10. 10. 10. 10. 10. 10. 10.
 10. 10. 10. 10. 10. 10. 10. 10. 10. 10. 10. 10. 10. 10. 10. 10. 10. 10.
 10. 10. 10. 10. 10. 14. 10. 10. 10. 10. 10. 10. 10. 10. 10. 10. 10. 10.
 10. 10. 10. 10. 10. 10. 10. 10. 10. 10. 10.]
0.26153846153846155
